# Exercise 2: Query Expansion and Indexing

The following code is modified from Exercise 1. It is used to construct the vocabulary and vectorize the documents and query.

In [3]:

from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import numpy as np

stemmer = PorterStemmer()

# Tokenize, stem a document
def tokenize(text):
    text = "".join([ch for ch in text if ch not in string.punctuation])
    tokens = nltk.word_tokenize(text)
    return " ".join([stemmer.stem(word.lower()) for word in tokens])

# Read a list of documents from a file. Each line in a file is a document
with open("bread.txt") as f:
# with open("epfldocs.txt") as f:
    content = f.readlines()
original_documents = [x.strip() for x in content] 
documents = [tokenize(d).split() for d in original_documents]

# create the vocabulary
vocabulary = set([item for sublist in documents for item in sublist])
vocabulary = [word for word in vocabulary if word not in stopwords.words('english')]
vocabulary.sort()

# compute IDF, storing idf values in a dictionary
def idf_values(vocabulary, documents):
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
        idf[term] = math.log(num_documents/sum(term in document for document in documents), math.e)
    return idf

# Function to generate the vector for a document (with normalisation)
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    max_count = counts.most_common(1)[0][1]
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]/max_count
    return vector

# Function to compute cosine similarity
def cosine_similarity(v1,v2):
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
        x = v1[i]; y = v2[i]
        sumxx += x*x
        sumyy += y*y
        sumxy += x*y
    if sumxy == 0:
            result = 0
    else:
            result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def vectorize_query(query, vocabulary, idf):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    return query_vector
    
def search_vec(query, k):
    query_vector = vectorize_query(query, vocabulary, idf)
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(documents))]
    scores.sort(key=lambda x: -x[0])
    ans = []
    indices = []
    for i in range(k):
        ans.append(original_documents[scores[i][1]])
        indices.append(scores[i][1])
    return ans, indices, query_vector

# Compute IDF values and vectors
idf = idf_values(vocabulary, documents)
document_vectors = [vectorize(s, vocabulary, idf) for s in documents]

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aberer/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Question 1 - Relevance Feedback

In this exercise we will implement and test Rocchio's method for user relevance feedback.

Let the set of relevant documents to be $D_r $ and the set of non-relevant documents to be $D_{nr}$. Then the modified query  $\vec{q_m}$  according to the Rocchio method is:

\begin{equation}
\vec{q_m} = \alpha \vec{q_0} + \frac{\beta}{|D_r|} \sum_{\vec{d_j} \in D_r} \vec{d_j} - \frac{\gamma}{|D_{nr}|} \sum_{\vec{d_j} \in D_{nr}} \vec{d_j}
\end{equation}
In the Rocchio algorithm negative term weights are ignored. This means, for the negative term weights in $\vec{q_m}$, we set them to be 0.

First, complete the implementation of the Rocchio relevance feedback method, by adding the missing code for the function $expand\_query$.   

Then, compare the result obtained with the new query with the unmodified one.

In [19]:
def expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, alpha, beta, gamma):
    
    num_rel = len(relevant_doc_vecs)
    num_non_rel = len(non_relevant_doc_vecs)
    
    # Compute the first term in the Rocchio equation
    norm_query_vector = ...
    
    # Compute the second term in the Rocchio equation
    norm_sum_relevant = ...
    
    # Compute the last term in the Rocchio equation
    norm_sum_non_relevant = ...
    
    # Sum all the terms
    modified_query_vector = ...
    
    # Ignore negative elements
    modified_query_vector = ...
    return modified_query_vector

The following code will give you the result for the query "computer science"


In [22]:
ans, result_doc_ids, query_vector = search_vec("computer science", 10)
for i in range(len(ans)):
    print(i,ans[i])

0 Exciting News: "World University Rankings 2016-2017 by subject: computer science" No1 @ETH &amp; @EPFL on No8. Congrats https://t.co/ARSlXZoShQ
1 New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD
2 An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj
3 New at @epfl_en Life Sciences @epflSV: "From PhD directly to Independent Group Leader" #ELFIR_EPFL:  Early Independence Research Scholars. See https://t.co/evqyqD7FFl, also for computational biology #compbio https://t.co/e3pDCg6NVb Deadline April 1 2018 at https://t.co/mJqcrfIqkb
4 Video of Nicola Marzari from @EPFL_en  on Computational Discovery in the 21st Century during #PASC17 now online: https://t.co/tfCkEvYKtq https://t.co/httPdHcK9W
5 Exposure Science Film Hackathon 2017 applications open! Come join our Scicomm-film-hacking event! #Science #scicomm https://t.co

The following code produces the result for the unmodified query (example: "computer science").

In [24]:
# list of indices marked as relevant

relevant_indices = [1,2]

# remove relevant indices from all indices
non_relevant_indices =  [n for i, n in enumerate([l for l in range(len(ans))]) if i not in relevant_indices]

def items_from_list(indices, alist):
    from operator import itemgetter
    return list(itemgetter(*indices)(alist))

relevant_doc_ids = items_from_list(relevant_indices, result_doc_ids)
non_relevant_doc_ids = items_from_list(non_relevant_indices, result_doc_ids)

relevant_doc_vecs = items_from_list(relevant_doc_ids, document_vectors)
non_relevant_doc_vecs = items_from_list(non_relevant_doc_ids, document_vectors)

expanded_query = expand_query(relevant_doc_vecs, non_relevant_doc_vecs, query_vector, 1, 1, 1)

new_query = ' '.join([vocabulary[i] for i, val in enumerate(expanded_query) if val>0])

new_ans , _, _ = search_vec(new_query, 10)

print('Modified query: ', new_query)
new_ans

Modified query:  barth biophys combin comput control distanc epflen httpstcob9tglxo4kd httpstcoijwbaebocj httpstcoznjk3bz6mo interview model new patrick professor protein scienc show vdtech via


['New computer model shows how proteins are controlled "at a distance" https://t.co/zNjK3bZ6mO  via @EPFL_en #VDtech https://t.co/b9TglXO4KD',
 'An interview with Patrick Barth, a new @EPFL professor who combines protein #biophysics with computer modeling https://t.co/iJwBaEbocj',
 'Interview (in French) de Patrick Aebischer, un "innovation slasher" @EPFL_en https://t.co/BtzhxEAEmN',
 'The proteins that domesticated our genomes https://t.co/npGbUKJhBl  via @EPFL_en #VDtech https://t.co/It0SBqlKQc',
 "New software can model natural light from the occupants' perspective https://t.co/RbMmN3Po5v via @EPFL_en #VDtech https://t.co/50enZtwUHi",
 "New software can model natural light from the occupants' perspective https://t.co/RbMmN3Po5v via @EPFL_en #VDtech https://t.co/lLIAvntc9R",
 'Latest work in our lab shows how feedback enhances brainwave control of a novel hand-exoskeleton https://t.co/VVPdX19fIM #epfl',
 '“Artificial intelligence has the potential to revolutionize transportation" int

## Question 2 -  Distributed Information Retrieval

In this exercise we implement a core process of Fagin's algorithm, the parallel scanning of the posting lists. Assume an aggregation function that returns the sum of the tf-idf scores given the terms in a document.

We assume that the posting lists for each term of a retrieval system are running on a different node.

We first create a dictionary $h$, where each entry of the dictionary corresponds to a posting list for term, assumed to be hosted in a different node. Explore the structure of $h$, to understand it. We suggest to first use the simple collection breads.txt.

In [55]:
import numpy as np
import operator

doc_vecs = np.transpose(np.array(document_vectors))
h = {}
for i, term in enumerate(vocabulary):
    ha = {}
    for docj in range(len(original_documents)):
        tfidf = doc_vecs[i][docj]
        ha[docj] = tfidf
    sorted_ha = sorted(ha.items(), key=operator.itemgetter(1), reverse=True)
    h[term] = sorted_ha


### Complete the following function that implements the Fagin algorithm.

In [93]:
def fagin_algorithm(query, h, k, vocabulary):
    
    # Split and stem the query
    q = query.split()
    q = set([stemmer.stem(w) for w in q])
    query_term_cnt = len(q)
    
    # select the posting lists for the query terms
    posting_lists = {}
    for term in q:
        if term in h:
            posting_lists[term] = h[term]
    
    max_len = len(documents)
    
    # Traverse the selected posting lists until we found k documents that appear in ALL posting lists
    # This corresponds to phase 1 of Fagin's algorithm.
    # As a result you produce a dictionary documents_occurrences, with the document identifiers as keys, 
    # and the number of documents found as value.
    # We stop traversing the posting lists until we have found k documents that appear in ALL posting lists 
    # of the query terms
    
    documents_occurrences = {}

    ...
                
    print(documents_occurrences)
        
    # Retrieve for the found documents the tfidf values and add them up.
    # For simplicity, we do not distinguish the cases whether the values have already been retrieved in the 
    # previous phase, or whether we use random access to obtain those values
    
    tfidf = {}
    for d in documents_occurrences.keys():
        t = 0
        for term in q:
            t = t + dict(posting_lists[term])[d]
        tfidf[d] = t
        
    # Finally we compute the top-k documents and return the answer
    
    ans = sorted(tfidf.items(), key=lambda x:x[1], reverse = True)[:k]
    return ans


ans = fagin_algorithm("bread recipe", h, 2, vocabulary)
print(ans)
for document_id in ans:
    print(original_documents[document_id[0]])

{3: 2, 2: 1, 0: 2, 1: 1}
[(3, 1.1394342831883648), (0, 0.56971714159418241)]
Breads, Pastries, Pies, and Cakes: Quantity Baking Recipes
How to Bake Breads Without Baking Recipes


Produce your own datasets to explore the behavior of the algorithm. Create a dataset such that for retrieving a 2 term query a total of 6 documents needs to be retrieved in the first phase of the algorithm (as in the example in the lecture).

## Question 3 - Inverted Files (Hard)

We learnt in the lecture that terms are typically stored in an inverted list. Now, in the inverted list, instead of only storing document identifiers of the documents in which the term appears, assume we also store an *offset* of the appearance of a term in a document. An $offset$ of a term $l_k$ given a document is defined as the number of words between the start of the document and $l_k$. Thus our inverted list is now:

$l_k= \langle f_k: \{d_{i_1} \rightarrow [o_1,\ldots,o_{n_{i_1}}]\}, 
\{d_{i_2} \rightarrow [o_1,\ldots,o_{n_{i_2}}]\}, \ldots, 
\{d_{i_k} \rightarrow [o_1,\ldots,o_{n_{i_k}}]\} \rangle$

This means that in document $d_{i_1}$ term $l_k$ appears $n_{i_1}$ times and at offset $[o_1,,o_{n_{i_1}}]$, where $[o_1,,o_{n_{i_1}}]$ are sorted in ascending order, these type of indices are also known as term-offset indices. An example of a term-offset index is as follows:

**Obama** = $⟨4 : {1 → [3]},{2 → [6]},{3 → [2,17]},{4 → [1]}⟩$

**Governor** = $⟨2 : {4 → [3]}, {7 → [14]}⟩$

**Election** = $⟨4 : {1 → [1]},{2 → [1,21]},{3 → [3]},{5 → [16,22,51]}⟩$

Which is to say that the term **Governor** appear in 2 documents. In document 4 at offset 3, in document 7 at offset 14. Now let us consider the *SLOP/x* operator in text retrieval. This operator has the syntax: *QueryTerm1 SLOP/x QueryTerm2* finds occurrences of *QueryTerm1* within $x$ (but not necessarily in that order) words of *QueryTerm2*, where $x$ is a positive integer argument ($x \geq 1$). Thus $x = 1$ demands that *QueryTerm1* be adjacent to *QueryTerm2*.

1. List the set of documents that satisfy the query **Obama** *SLOP/2* **Election**.
2. List each set of values for which the query **Obama** *SLOP/x* **Election** has a different set of documents as answers (starting from $x = 1$). 
3. Consider the general procedure for "merging" two term-offset inverted lists for a given document, to determine where the document satisfies a *SLOP/x* clause (since in general there will be many offsets at which each term occurs in a document). Let $L$ denote the total number of occurrences of the two terms in the document. Assume we have a pointer to the list of occurrences of each term and can move the pointer along this list. As we do so we check whether we have a hit for $SLOP/x$ (i.e. the $SLOP/x$ clause is satisfied). Each move of either pointer counts as a step. Based on this assumption is there a general "merging" procedure to determine whether the document satisfies a $SLOP/x$ clause, for which the following is true? Justify your answer.

    1. The merge can be accomplished in a number of steps linear in $L$ regardless of $x$, and we can ensure that each pointer moves only to the right (i.e. forward).
    2. The merge can be accomplished in a number of steps linear in $L$, but a pointer may be forced to move to the left (i.e. backwards).
    3. The merge can require $x \times L$ steps in some cases.